In [ ]:
from pandas_datareader import data
import mplfinance as mpf
import pandas

import tkinter as tk
import tkinter.ttk as ttk
import datetime
import time
import csv
import os
import sys
from PIL import Image
#数値を画面に出力するための関数
csvfile_path = './stocklist.csv'
df = pandas.read_csv(csvfile_path)

def draw_graph():
    #テキストボックスの内容を取得
    global number, start, end
    number2=str(number.get())+".JP"
    start2 =str(start.get())
    end2   =str(end.get())
    filename = number2+'.png'
    #株式データの取得
    df = data.DataReader(number2, 'stooq', start2, end2)
    df = df.sort_index()
    #グラフの描画と保存
    mpf.plot(df, title='\n'+number2, type='candle', mav=(5, 25), volume=True, savefig=filename)
 
    #画像のサイズ取得
    img = Image.open(filename)
    width = img.width
    height = img.height
 
    #サブウィンドウの追加
    sub = tk.Toplevel()
    sub.title(number2)
    sub.geometry(str(width)+"x"+str(height))
    #グラフの描画
    graph = tk.PhotoImage(file=filename)
    canvas = tk.Canvas(sub,bg='white', width = width, height = height)
    canvas.place(x=0, y=0)
    canvas.create_image(0,0,image=graph, anchor=tk.NW)

#数値を画面に出力するための関数
def search_table():
    global name, df
    name2=str(name.get())
    df_part = df[df['銘柄名'].str.contains(name2)]
    df_part_str=df_part.astype(str)
    #サブウィンドウの追加
    sub = tk.Toplevel()
    sub.title("search")
    sub.geometry("500x500")
    #グラフの描画
    search_label = ttk.Label(
        sub,
        text=df_part_str,
        width=50,
        relief=tk.SUNKEN,
    )
    search_label.grid(row=0, column=0, padx=20, pady=5)

# トップレベルウィンドウ作成
root = tk.Tk()


name_label = ttk.Label(
    root,
    text="name",
    width=5,
    relief=tk.SUNKEN,
)
name_label.grid(row=0, column=0, padx=20, pady=5)

name = tk.Entry(root,width=20)
name.grid(row=0, column=1, padx=10, pady=5)

get = tk.StringVar(root)
get.set("GET!")
get_button = tk.Button(root, textvariable=get, command=search_table)
get_button.grid(row=0, column=3, padx=10, pady=5)
    

number_label = ttk.Label(
    root,
    text="num",
    width=5,
    relief=tk.SUNKEN,
)
number_label.grid(row=1, column=0, padx=20, pady=5)

start_label = ttk.Label(
    root,
    text="start",
    width=5,
    relief=tk.SUNKEN,
)
start_label.grid(row=2, column=0, padx=20, pady=5)

end_label = ttk.Label(
    root,
    text="end",
    width=5,
    relief=tk.SUNKEN,
)
end_label.grid(row=3, column=0, padx=20, pady=5)


number = tk.Entry(width=20)
number.grid(row=1, column=1, padx=10, pady=5)
start = tk.Entry(width=20)
start.grid(row=2, column=1, padx=10, pady=5)
end = tk.Entry(width=20)
end.grid(row=3, column=1, padx=10, pady=5)



draw = tk.StringVar(root)
draw.set("DRAW!")
draw_button = tk.Button(root, textvariable=draw, command=draw_graph)
draw_button.grid(row=4, column=1, padx=10, pady=5)

root.mainloop()